In [ ]:
!pip install efficientnet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=09d87f95e9a08927a2ab14e5cdf8123351c3337f1c6358dea585a6cf19727705
  Stored in directory: /root/.cache/pip/wheels/84/b9/90/25a0195cf95fb5533db96f1c77ea3f296b7cc86ae8ae48e3dc
Successfully built efficientnet-pytorch


In [ ]:
!pip install wandb

#4. 패키지 import
import random
from copy import deepcopy
import math
import time
import pandas as pd
import numpy as np
import os
from skimage import io
from PIL import Image
import wandb
import tqdm
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as T
import matplotlib.pyplot as plt
from sklearn import metrics
import importlib
from torchsummary import summary
from efficientnet_pytorch import EfficientNet

torch.multiprocessing.set_start_method('spawn')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.6 MB/s 
     |████████████████████████████████| 168 kB 92.5 MB/s 
     |████████████████████████████████| 182 kB 83.2 MB/s 
     |████████████████████████████████| 62 kB 822 kB/s 
     |████████████████████████████████| 168 kB 75.6 MB/s 
     |████████████████████████████████| 166 kB 72.8 MB/s 
     |████████████████████████████████| 166 kB 91.9 MB/s 
     |████████████████████████████████| 162 kB 96.6 MB/s 
     |████████████████████████████████| 162 kB 78.4 MB/s 
     |████████████████████████████████| 158 kB 82.4 MB/s 
     |████████████████████████████████| 157 kB 80.3 MB/s 
     |████████████████████████████████| 157 kB 105.4 MB/s 
     |████████████████████████████████| 157 kB 105.1 MB/s 
     |████████████████████████████████| 157 kB 108.7 MB/s 
     |████████████████████████████████| 157 kB 97.1 MB/s 
     |█████████████████████████

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
x_data = {'train':[],'val':[]}
y_data = {'train':[],'val':[]}
len_data = {'train':617, 'val':108}
for phase in ['train','val']:
    for i in tqdm.tqdm(range(len_data[phase])):
        if phase == 'train' and len(torch.load(f'/content/drive/MyDrive/final_classification_dataset/y_data_{i}_{phase}.pth')) == 0:
            continue
        x_data[phase].append(f'/content/drive/MyDrive/final_classification_dataset/x_data_{i}_{phase}.pth')
        y_data[phase].append(f'/content/drive/MyDrive/final_classification_dataset/y_data_{i}_{phase}.pth')


100%|██████████| 108/108 [00:00<00:00, 522473.85it/s]


In [ ]:
class MyDataSet2(data.Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
    def __getitem__(self, index):
        img = torch.load(self.x_data[index]).float()
        #img = self.x_data[index]
        if index%2 == 0:
            img = torch.flip(img, [3])
        target = torch.load(self.y_data[index]).float()
        a = np.where(target == 1)
        if a[0][-1] == 4:
            target_ = torch.tensor([0,0,0,1])
        else:
            target_ = torch.tensor([1 if i==a[0][-1] else 0 for i in range(4)])
        return img, target_.type(torch.float32)
    def __len__(self):
        return len(self.x_data)

dataset_train = MyDataSet2(x_data['train'],y_data['train'])
dataset_val = MyDataSet2(x_data['val'],y_data['val'])

# pipeline

In [ ]:
!pip install git+https://github.com/shijianjian/EfficientNet-PyTorch-3D

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shijianjian/EfficientNet-PyTorch-3D to /tmp/pip-req-build-egyqbvkz
  Running command git clone -q https://github.com/shijianjian/EfficientNet-PyTorch-3D /tmp/pip-req-build-egyqbvkz
  Created wheel for efficientnet-pytorch-3d: filename=efficientnet_pytorch_3d-0.6.3-py3-none-any.whl size=16770 sha256=c4018a470a1b0755882f3d7adc93240f4b317d8a3243accefe29b25a424b691a
  Stored in directory: /tmp/pip-ephem-wheel-cache-34z__sv_/wheels/71/7b/d7/7845882d34b3ee447d8940f660a12da4c88a077f55c45c73ac
Successfully built efficientnet-pytorch-3d


In [ ]:
from torch.nn.modules.container import ModuleList
from efficientnet_pytorch_3d import EfficientNet3D
import torch

model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 4}, in_channels=3)
model = model.cuda()

from torchsummary import summary
summary(model, input_size=( 3, 160, 160, 160))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1     [-1, 3, 161, 161, 161]               0
Conv3dStaticSamePadding-2       [-1, 32, 80, 80, 80]           2,592
       BatchNorm3d-3       [-1, 32, 80, 80, 80]              64
MemoryEfficientSwish-4       [-1, 32, 80, 80, 80]               0
         ZeroPad2d-5       [-1, 32, 81, 81, 81]               0
Conv3dStaticSamePadding-6       [-1, 32, 40, 40, 40]             864
       BatchNorm3d-7       [-1, 32, 40, 40, 40]              64
MemoryEfficientSwish-8       [-1, 32, 40, 40, 40]               0
          Identity-9          [-1, 32, 1, 1, 1]               0
Conv3dStaticSamePadding-10           [-1, 8, 1, 1, 1]             264
MemoryEfficientSwish-11           [-1, 8, 1, 1, 1]               0
         Identity-12           [-1, 8, 1, 1, 1]               0
Conv3dStaticSamePadding-13          [-1, 32, 1, 1, 1]             288
         I

In [ ]:
config = dict(
    lr_in  = 0.001,
    lr_out = 0.001,
    momentum = 1,
    num_epochs = 100,
    power = -1,
    lamb = 1,
    batch_size = 16,
    batch_multiplier = 3,
    lossfunc = 'cross_entropy', #SGD, 다른 loss function 찾아보기
    gradient = 'Adam',
    architecture="ref_3DResNet18")

In [ ]:
with wandb.init(project='sinusitis_renewal_efficientnet_b0_new_data_augmentation', config=config):
    wandb.run.name = 'efficient_net_lr=0.001'
    model = model.cuda()
    inputs = torch.randn((1, 3, 160, 160, 160)).cuda()
    labels = torch.tensor([0]).cuda()
    # test forward
    num_classes = 2

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    device = 'cuda'
    batch_size=16

    train_dataloader = data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    val_dataloader = data.DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

    dataset = {'train':train_dataloader, 'val':val_dataloader}



    for epoch in range(100): 
        num_correct_img = 0.0
        num_total_img = 0
        split_train_loss = 0.0
        split_val_loss = 0.0
        t_split_start = time.time()
        t_iter_start = time.time()
        tot_output=torch.tensor([])
        tot_target=torch.tensor([])
        iteration = 1
        num_train_imgs = len(dataset_train)
        num_val_imgs = len(dataset_val)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 훈련 모드로
                optimizer.zero_grad()
                print('(train)')
            else:
                model.eval()   # 모델을 검증 모드로
                print('-------------')
                print('(val)')
            # 데이터 로더에서 minibatch씩 꺼내 루프
            count = 0  # multiple minibatch
            for imges, anno_class_imges in tqdm.tqdm(dataset[phase], desc = f'{epoch+1}th epoch'):
                # 미니배치 크기가 1이면 배치 노멀라이제이션에서 오류가 발생하므로 회피
                if imges.size()[0] == 1:
                    continue
                # GPU가 사용가능하면 GPU에 데이터를 보낸다
                imges = imges.to(device)
                anno_class_imges = anno_class_imges.to(device)
                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(imges)
                    loss = criterion(
                        outputs, anno_class_imges)
                    #print(outputs.cpu().detach().numpy())
                    #print(anno_class_imges.cpu().detach().numpy()
                    if phase == 'train':
                        loss.backward()  # 경사 계산
                        optimizer.step()
                        count -= 1  # multiple minibatc
                        if (iteration % 10 == 0) or iteration == 38:  # 10iter에 한 번, loss를 표시
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('반복 {} || cross-entropy loss: {:4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size, duration))
                            t_iter_start = time.time()
                        split_train_loss += loss.item()
                        iteration += 1
                    # 검증 시
                    else:
                        outputs = F.softmax(outputs,dim=1)
                        output = outputs>0.4
                        output = output.to('cpu')
                        anno_class_imges = anno_class_imges.to('cpu')
                        print(outputs, anno_class_imges)
                        tot_output=torch.cat((tot_output, output))
                        tot_target=torch.cat((tot_target, anno_class_imges))
                        correct = (((output == anno_class_imges).sum(dim=1))==5).sum().item()
                        num_correct_img += correct
                        num_total_img += len(output)
                        split_val_loss += loss.item()

                # if(iteration > 50):
                #     torch.save(net.state_dict(), 'weights/test.pth')
                #     break
                del imges
                del anno_class_imges
                torch.cuda.empty_cache()
        acc = num_correct_img/num_total_img
        log_split = {'epoch':epoch+1, 'split_train_loss': split_train_loss /
        num_train_imgs, 'split_val_loss': split_val_loss/num_val_imgs, 'accuracy': acc}
        wandb.log(log_split)
        torch.save({'epoch':epoch, 'model_state_dict':model.state_dict(), 'optimizer_state_dict':optimizer.state_dict()}, '/content/drive/MyDrive/weights and log2/efficientnet_non_seg'+ f'_{epoch}'+'.pth')
        torch.save({'checkpoint': '/content/drive/MyDrive/weights and log2/efficientnet_non_seg'+ f'_{epoch}'+'.pth'}, '/content/drive/MyDrive/weights and log2/non_seg_route_1130.pth')
        print('최후의 네트워크 저장 완료')



print('Finished Training')

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: triangle756. Use `wandb login --relogin` to force relogin


(train)


1th epoch:  26%|██▋       | 10/38 [02:17<06:05, 13.04s/it]

반복 10 || cross-entropy loss: 0.081067 || 10iter: 137.7128 sec.


1th epoch:  53%|█████▎    | 20/38 [04:29<03:54, 13.02s/it]

...

100th epoch: 100%|██████████| 7/7 [01:04<00:00,  9.20s/it]

tensor([[0.0065, 0.5488, 0.4053, 0.0395],
        [0.0153, 0.4871, 0.4665, 0.0312],
        [0.0097, 0.5177, 0.4456, 0.0271],
        [0.0062, 0.5509, 0.3858, 0.0571],
        [0.0042, 0.5760, 0.3847, 0.0351],
        [0.0153, 0.4883, 0.4615, 0.0350],
        [0.0033, 0.5907, 0.3705, 0.0354],
        [0.0083, 0.5297, 0.4318, 0.0302],
        [0.0474, 0.3739, 0.4100, 0.1687],
        [0.0072, 0.5366, 0.3783, 0.0779],
        [0.0104, 0.5108, 0.4553, 0.0235],
        [0.0040, 0.5795, 0.3750, 0.0414]], device='cuda:0') tensor([[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]])


최후의 네트워크 저장 완료


accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
split_train_loss,▃█▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_val_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.0
epoch,100
split_train_loss,0.05794
split_val_loss,0.19348


Finished Training


In [ ]:
x_data = {'train':[],'val':[]}
y_data = {'train':[],'val':[]}
len_data = {'train':617, 'val':108}
for phase in ['train','val']:
    for i in tqdm.tqdm(range(len_data[phase])):
        if phase == 'train' and len(torch.load(f'/content/drive/MyDrive/final_classification_dataset_seg/y_data_{i}_{phase}.pth')) == 0:
            continue
        x_data[phase].append(f'/content/drive/MyDrive/final_classification_dataset_seg/x_data_{i}_{phase}.pth')
        y_data[phase].append(f'/content/drive/MyDrive/final_classification_dataset_seg/y_data_{i}_{phase}.pth')


100%|██████████| 108/108 [00:00<00:00, 489713.33it/s]


In [ ]:
class MyDataSet2(data.Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
    def __getitem__(self, index):
        img = torch.load(self.x_data[index]).float()
        #img = self.x_data[index]
        if index%2 == 0:
            img = torch.flip(img, [3])
        target = torch.load(self.y_data[index]).float()
        a = np.where(target == 1)
        if a[0][-1] == 4:
            target_ = torch.tensor([0,0,0,1])
        else:
            target_ = torch.tensor([1 if i==a[0][-1] else 0 for i in range(4)])
        return img, target_.type(torch.float32)
    def __len__(self):
        return len(self.x_data)

dataset_train = MyDataSet2(x_data['train'],y_data['train'])
dataset_val = MyDataSet2(x_data['val'],y_data['val'])

In [ ]:
from torch.nn.modules.container import ModuleList
from efficientnet_pytorch_3d import EfficientNet3D
import torch

model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 4}, in_channels=3)
model = model.cuda()

from torchsummary import summary
summary(model, input_size=( 3, 160, 160, 160))

with wandb.init(project='sinusitis_renewal_efficientnet_b0_new_data_augmentation', config=config):
    wandb.run.name = 'efficient_net_seg'
    model = model.cuda()
    inputs = torch.randn((1, 3, 160, 160, 160)).cuda()
    labels = torch.tensor([0]).cuda()
    # test forward
    num_classes = 2

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    device = 'cuda'
    batch_size=16

    train_dataloader = data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    val_dataloader = data.DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

    dataset = {'train':train_dataloader, 'val':val_dataloader}



    for epoch in range(100): 
        num_correct_img = 0.0
        num_total_img = 0
        split_train_loss = 0.0
        split_val_loss = 0.0
        t_split_start = time.time()
        t_iter_start = time.time()
        tot_output=torch.tensor([])
        tot_target=torch.tensor([])
        iteration = 1
        num_train_imgs = len(dataset_train)
        num_val_imgs = len(dataset_val)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 훈련 모드로
                optimizer.zero_grad()
                print('(train)')
            else:
                model.eval()   # 모델을 검증 모드로
                print('-------------')
                print('(val)')
            # 데이터 로더에서 minibatch씩 꺼내 루프
            count = 0  # multiple minibatch
            for imges, anno_class_imges in tqdm.tqdm(dataset[phase], desc = f'{epoch+1}th epoch'):
                # 미니배치 크기가 1이면 배치 노멀라이제이션에서 오류가 발생하므로 회피
                if imges.size()[0] == 1:
                    continue
                # GPU가 사용가능하면 GPU에 데이터를 보낸다
                imges = imges.to(device)
                anno_class_imges = anno_class_imges.to(device)
                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(imges)
                    loss = criterion(
                        outputs, anno_class_imges)
                    #print(outputs.cpu().detach().numpy())
                    #print(anno_class_imges.cpu().detach().numpy()
                    if phase == 'train':
                        loss.backward()  # 경사 계산
                        optimizer.step()
                        count -= 1  # multiple minibatc
                        if (iteration % 10 == 0) or iteration == 38:  # 10iter에 한 번, loss를 표시
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('반복 {} || cross-entropy loss: {:4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size, duration))
                            t_iter_start = time.time()
                        split_train_loss += loss.item()
                        iteration += 1
                    # 검증 시
                    else:
                        outputs = F.softmax(outputs,dim=1)
                        output = outputs>0.4
                        output = output.to('cpu')
                        anno_class_imges = anno_class_imges.to('cpu')
                        print(outputs, anno_class_imges)
                        tot_output=torch.cat((tot_output, output))
                        tot_target=torch.cat((tot_target, anno_class_imges))
                        correct = (((output == anno_class_imges).sum(dim=1))==5).sum().item()
                        num_correct_img += correct
                        num_total_img += len(output)
                        split_val_loss += loss.item()

                # if(iteration > 50):
                #     torch.save(net.state_dict(), 'weights/test.pth')
                #     break
                del imges
                del anno_class_imges
                torch.cuda.empty_cache()
        acc = num_correct_img/num_total_img
        log_split = {'epoch':epoch+1, 'split_train_loss': split_train_loss /
        num_train_imgs, 'split_val_loss': split_val_loss/num_val_imgs, 'accuracy': acc}
        wandb.log(log_split)
        torch.save({'epoch':epoch, 'model_state_dict':model.state_dict(), 'optimizer_state_dict':optimizer.state_dict()}, '/content/drive/MyDrive/weights and log2/efficientnet_seg'+ f'_{epoch}'+'.pth')
        torch.save({'checkpoint': '/content/drive/MyDrive/weights and log2/efficientnet_seg'+ f'_{epoch}'+'.pth'}, '/content/drive/MyDrive/weights and log2/seg_route_1130.pth')
        print('최후의 네트워크 저장 완료')



print('Finished Training')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1     [-1, 3, 161, 161, 161]               0
Conv3dStaticSamePadding-2       [-1, 32, 80, 80, 80]           2,592
       BatchNorm3d-3       [-1, 32, 80, 80, 80]              64
MemoryEfficientSwish-4       [-1, 32, 80, 80, 80]               0
         ZeroPad2d-5       [-1, 32, 81, 81, 81]               0
Conv3dStaticSamePadding-6       [-1, 32, 40, 40, 40]             864
       BatchNorm3d-7       [-1, 32, 40, 40, 40]              64
MemoryEfficientSwish-8       [-1, 32, 40, 40, 40]               0
          Identity-9          [-1, 32, 1, 1, 1]               0
Conv3dStaticSamePadding-10           [-1, 8, 1, 1, 1]             264
MemoryEfficientSwish-11           [-1, 8, 1, 1, 1]               0
         Identity-12           [-1, 8, 1, 1, 1]               0
Conv3dStaticSamePadding-13          [-1, 32, 1, 1, 1]             288
         I

(train)


1th epoch:  26%|██▋       | 10/38 [01:51<04:58, 10.67s/it]

반복 10 || cross-entropy loss: 0.095673 || 10iter: 111.1411 sec.


1th epoch:  53%|█████▎    | 20/38 [03:44<03:27, 11.50s/it]

...

100th epoch: 100%|██████████| 7/7 [01:05<00:00,  9.41s/it]

tensor([[0.0363, 0.0182, 0.2455, 0.7000],
        [0.6508, 0.1824, 0.1009, 0.0658],
        [0.5920, 0.1864, 0.1317, 0.0900],
        [0.0364, 0.0186, 0.2496, 0.6953],
        [0.0267, 0.0123, 0.2120, 0.7490],
        [0.6146, 0.1836, 0.1200, 0.0818],
        [0.0231, 0.0103, 0.1977, 0.7689],
        [0.2625, 0.1489, 0.3191, 0.2695],
        [0.6332, 0.1847, 0.1099, 0.0722],
        [0.0243, 0.0112, 0.2063, 0.7583],
        [0.6692, 0.1825, 0.0912, 0.0571],
        [0.0444, 0.0243, 0.2749, 0.6564]], device='cuda:0') tensor([[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]])


최후의 네트워크 저장 완료


accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
split_train_loss,▂▇▆█▆▄▃▅▃▃▅▄▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_val_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.0
epoch,100
split_train_loss,0.06494
split_val_loss,0.05106


Finished Training
